In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install pandas==0.22.0
!pip install xgboost==1.0.0

In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#import lime
#import lime.lime_tabular
#from lime import submodular_pick;
import warnings
warnings.filterwarnings('ignore')

import joblib

In [8]:
dataset_ref = "bpic2011"
params_dir = PATH + "params/"
results_dir = "results"
bucket_method = "single"
cls_encoding = "agg"
cls_method = "xgboost"
gap = 1
n_iter = 1

if bucket_method == "state":
    bucket_encoding = "last"
else:
    bucket_encoding = "agg"

method_name = "%s_%s"%(bucket_method, cls_encoding)

dataset_ref_to_datasets = {
    "bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

encoding_dict = {
    "laststate": ["static", "last"],
    "agg": ["static", "agg"],
    "index": ["static", "index"],
    "combined": ["static", "last", "agg"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]
    
train_ratio = 0.8
random_state = 22

# create results directory
if not os.path.exists(os.path.join(params_dir)):
    os.makedirs(os.path.join(params_dir))
    
print(datasets)

['bpic2011_f1', 'bpic2011_f2', 'bpic2011_f3', 'bpic2011_f4']


In [9]:
for dataset_name in datasets:
    
    # load optimal params
    print("Setting up parameters...")
    optimal_params_filename = os.path.join(params_dir, "optimal_params_%s_%s_%s.pickle" % (cls_method, dataset_name, method_name))

    if not os.path.isfile(optimal_params_filename) or os.path.getsize(optimal_params_filename) <= 0:
        print("Parameters not found")
        
    with open(optimal_params_filename, "rb") as fin:
        args = pickle.load(fin)
        
    #print(args)
            
    # read the data
    print("setting up data...")
    dataset_manager = DatasetManager(dataset_name)
    data = dataset_manager.read_dataset()
    #print('Case ID column', dataset_manager.case_id_col)
    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

    # determine min and max (truncated) prefix lengths
    min_prefix_length = 1
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    elif "bpic2017" in dataset_name:
        max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
        max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

    # split into training and test
    train, test = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
    
    if gap > 1:
        outfile = os.path.join(results_dir, "performance_results_%s_%s_%s_gap%s.csv" % (cls_method, dataset_name, method_name, gap))
    else:
        outfile = os.path.join(results_dir, "performance_results_%s_%s_%s.csv" % (cls_method, dataset_name, method_name))
        
    start_test_prefix_generation = time.time()
    dt_test_prefixes = dataset_manager.generate_prefix_data(test, min_prefix_length, max_prefix_length)
    test_prefix_generation_time = time.time() - start_test_prefix_generation
            
    offline_total_times = []
    online_event_times = []
    train_prefix_generation_times = []

    for ii in range(n_iter):
        # create prefix logs
        print("Creating logs...")
        start_train_prefix_generation = time.time()
        dt_train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length, gap)
        train_prefix_generation_time = time.time() - start_train_prefix_generation
        train_prefix_generation_times.append(train_prefix_generation_time)
            
        # Bucketing prefixes based on control flow
        print("bucketing prefixes...")
        bucketer_args = {'encoding_method':bucket_encoding, 
                         'case_id_col':dataset_manager.case_id_col, 
                         'cat_cols':[dataset_manager.activity_col], 
                         'num_cols':[], 
                         'random_state':random_state}
        if bucket_method == "cluster":
            bucketer_args["n_clusters"] = int(args["n_clusters"])
        bucketer = BucketFactory.get_bucketer(bucket_method, **bucketer_args)

        start_offline_time_bucket = time.time()
        bucket_assignments_train = bucketer.fit_predict(dt_train_prefixes)
        offline_time_bucket = time.time() - start_offline_time_bucket

        bucket_assignments_test = bucketer.predict(dt_test_prefixes)

        preds_all = []
        probas_all = []
        pred_types_all = []
        test_y_all = []
        nr_events_all = []
        offline_time_fit = 0
        current_online_event_times = []
        
        for bucket in set(bucket_assignments_test):
            print("Bucket" , bucket )
            print("sorting bucket...")
            if bucket_method == "prefix":
                current_args = args[bucket]
            else:
                current_args = args
            relevant_train_cases_bucket = dataset_manager.get_indexes(dt_train_prefixes)[bucket_assignments_train == bucket]
            relevant_test_cases_bucket = dataset_manager.get_indexes(dt_test_prefixes)[bucket_assignments_test == bucket]
            dt_test_bucket = dataset_manager.get_relevant_data_by_indexes(dt_test_prefixes, relevant_test_cases_bucket)
            
            nr_events_all.extend(list(dataset_manager.get_prefix_lengths(dt_test_bucket)))
            print('number events', len(nr_events_all))
            
            if len(relevant_train_cases_bucket) == 0:
                preds = [dataset_manager.get_class_ratio(train)] * len(relevant_test_cases_bucket)
                current_online_event_times.extend([0] * len(preds))
            else:
                dt_train_bucket = dataset_manager.get_relevant_data_by_indexes(dt_train_prefixes, relevant_train_cases_bucket) # one row per event
                train_y = dataset_manager.get_label_numeric(dt_train_bucket)

                if len(set(train_y)) < 2:
                    preds = [train_y[0]] * len(relevant_test_cases_bucket)
                    current_online_event_times.extend([0] * len(preds))
                    test_y_all.extend(dataset_manager.get_label_numeric(dt_test_bucket))
                else:
                    print("choosing classifier...")
                    start_offline_time_fit = time.time()
                    feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])

                    if cls_method == "rf":
                        cls = RandomForestClassifier(n_estimators=500,
                                                     max_features=current_args['max_features'],
                                                     random_state=random_state)

                    elif cls_method == "xgboost":
                        cls = xgb.XGBClassifier(objective='binary:logistic',
                                                n_estimators=500,
                                                learning_rate= current_args['learning_rate'],
                                                subsample=current_args['subsample'],
                                                max_depth=int(current_args['max_depth']),
                                                colsample_bytree=current_args['colsample_bytree'],
                                                min_child_weight=int(current_args['min_child_weight']),
                                                seed=random_state)

                    elif cls_method == "logit":
                        cls = LogisticRegression(C=2**current_args['C'],
                                                 random_state=random_state)

                    elif cls_method == "svm":
                        cls = SVC(C=2**current_args['C'],
                                  gamma=2**current_args['gamma'],
                                  random_state=random_state)

                    if cls_method == "svm" or cls_method == "logit":
                        pipeline = Pipeline([('encoder', feature_combiner), ('scaler', StandardScaler()), ('cls', cls)])
                    else:
                        pipeline = Pipeline([('encoder', feature_combiner), ('cls', cls)])

                    print("fitting pipeline...")
                    pipeline.fit(dt_train_bucket, train_y)
                    
                    offline_time_fit += time.time() - start_offline_time_fit

                    # predict separately for each prefix case
                    preds = []
                    probas = []
                    pred_types = []
                    
                    test_all_grouped = dt_test_bucket.groupby(dataset_manager.case_id_col)
                    print("test data shape", dt_test_bucket.shape)
                    count_d=0 # count for deviant
                    count_r=0 #count for regular

                    tp_list = []
                    tn_list = []
                    fp_list = []
                    fn_list = []

                    iteration = 1
                    
                    for _, group in test_all_grouped:
                        print ("Testing instance", iteration, 'of', len(test_all_grouped))
                        test_y_group = dataset_manager.get_label_numeric(group)
                        test_y_all.extend(test_y_group)
                            
                        start = time.time()
                        _ = bucketer.predict(group)
                        
                        if cls_method == "svm":
                            pred = pipeline.decision_function(group)
                        else:
                          preds_pos_label_idx = np.where(cls.classes_ == 1)[0][0]
                          pred = pipeline.predict(group)[0]
                          proba = pipeline.predict_proba(group)[:,preds_pos_label_idx]
                        pipeline_pred_time = time.time() - start
                        current_online_event_times.append(pipeline_pred_time / len(group))
                        case_id = dataset_manager.get_case_ids(group)[0]
                        
                        if test_y_group == pred & pred == 0:
                          pred_type = 'TN'

                          instance_dict = {}
                          instance_dict['caseID'] = case_id
                          instance_dict['input'] = group
                          instance_dict['actual'] = test_y_group[0]
                          instance_dict['predicted'] = pred
                          instance_dict['proba'] = 1 - proba[0]
                          instance_dict['nr_events'] = dataset_manager.get_prefix_lengths(group)[0]
                          instance_dict['pred_type'] = pred_type

                          tn_list.append(instance_dict)

                        elif test_y_group == pred & pred == 1:
                          pred_type = 'TP'
                                            
                          instance_dict = {}
                          instance_dict['caseID'] = case_id
                          instance_dict['input'] = group
                          instance_dict['actual'] = test_y_group[0]
                          instance_dict['predicted'] = pred
                          instance_dict['proba'] = proba[0]
                          instance_dict['nr_events'] = dataset_manager.get_prefix_lengths(group)[0]
                          instance_dict['pred_type'] = pred_type

                          tp_list.append(instance_dict)

                        elif test_y_group != pred & pred == 0:
                          pred_type = 'FN'
                          
                          instance_dict = {}
                          instance_dict['caseID'] = case_id
                          instance_dict['input'] = group
                          instance_dict['actual'] = test_y_group[0]
                          instance_dict['predicted'] = pred
                          instance_dict['proba'] = 1 - proba[0]
                          instance_dict['nr_events'] = dataset_manager.get_prefix_lengths(group)[0]
                          instance_dict['pred_type'] = pred_type

                          fn_list.append(instance_dict)

                        elif test_y_group != pred & pred == 1:
                          pred_type = 'FP'
                          
                          instance_dict = {}
                          instance_dict['caseID'] = case_id
                          instance_dict['input'] = group
                          instance_dict['actual'] = test_y_group[0]
                          instance_dict['predicted'] = pred
                          instance_dict['proba'] = proba[0]
                          instance_dict['nr_events'] = dataset_manager.get_prefix_lengths(group)[0]
                          instance_dict['pred_type'] = pred_type

                          fp_list.append(instance_dict)
                        else:
                          print("ERROR: Prediction doesn't fit into any category. Check code.")
                              
                        preds.append(pred)
                        probas.extend(proba)
                        pred_types.append(pred_type)
                        iteration+=1
                        
                    preds_all.extend(preds)
                    probas_all.extend(probas)
                    pred_types_all.extend(pred_types)

                    tn_path = os.path.join(PATH, "%s/%s_%s/instances/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucket))
                    tp_path = os.path.join(PATH, "%s/%s_%s/instances/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucket))
                    fn_path = os.path.join(PATH, "%s/%s_%s/instances/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucket))
                    fp_path = os.path.join(PATH, "%s/%s_%s/instances/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucket))
                    
                    with open (tn_path, 'wb') as f:
                      pickle.dump(tn_list, f)
                    with open (tp_path, 'wb') as f:
                      pickle.dump(tp_list, f)
                    with open (fn_path, 'wb') as f:
                      pickle.dump(fn_list, f)
                    with open (fp_path, 'wb') as f:
                      pickle.dump(fp_list, f)

            #Save models and encoders
            print('Saving models...')
            pipeline_path = os.path.join(PATH, "%s/%s_%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, dataset_name, cls_method, method_name, bucket))
            feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucket))
            bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucket))
            cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucket))
            joblib.dump(pipeline, pipeline_path)
            joblib.dump(feature_combiner, feat_comb_path)
            joblib.dump(cls, cls_path)
            joblib.dump(bucketer, bucketer_path)

            #Save training data
            print("saving data...")
            X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucket))
            Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucket))
            with open(X_train_path, 'wb') as f:
              pickle.dump(dt_train_bucket, f)
            with open(Y_train_path, 'wb') as f:
              pickle.dump(train_y, f)

            #Save testing data
            X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucket))
            Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucket))
            with open(X_test_path, 'wb') as f:
                pickle.dump(dt_test_bucket, f)
            with open(Y_test_path, 'wb') as f:
                pickle.dump(test_y_group, f)
        
        print("compiling results...")
        dt_results = pd.DataFrame({"actual": test_y_all, "predicted": preds_all, "nr_events": nr_events_all})
        for nr_events, group in dt_results.groupby("nr_events"):
            if len(set(group.actual)) < 2:
                print(dataset_name, method_name, cls_method, nr_events, "auc", np.nan)
            else:
                print(dataset_name, method_name, cls_method, nr_events, "auc", roc_auc_score(group.actual, group.predicted))
        print(dataset_name, method_name, cls_method, -1, -1, "auc", roc_auc_score(dt_results.actual, dt_results.predicted))


Setting up parameters...
setting up data...
Creating logs...
bucketing prefixes...
single
Bucket 1
sorting bucket...
number events 5889
choosing classifier...
fitting pipeline...
test data shape (92387, 26)
Testing instance 1 of 5889
Testing instance 2 of 5889
Testing instance 3 of 5889
Testing instance 4 of 5889
Testing instance 5 of 5889
Testing instance 6 of 5889
Testing instance 7 of 5889
Testing instance 8 of 5889
Testing instance 9 of 5889
Testing instance 10 of 5889
Testing instance 11 of 5889
Testing instance 12 of 5889
Testing instance 13 of 5889
Testing instance 14 of 5889
Testing instance 15 of 5889
Testing instance 16 of 5889
Testing instance 17 of 5889
Testing instance 18 of 5889
Testing instance 19 of 5889
Testing instance 20 of 5889
Testing instance 21 of 5889
Testing instance 22 of 5889
Testing instance 23 of 5889
Testing instance 24 of 5889
Testing instance 25 of 5889
Testing instance 26 of 5889
Testing instance 27 of 5889
Testing instance 28 of 5889
Testing instance 2

Testing instance 281 of 5889
Testing instance 282 of 5889
Testing instance 283 of 5889
Testing instance 284 of 5889
Testing instance 285 of 5889
Testing instance 286 of 5889
Testing instance 287 of 5889
Testing instance 288 of 5889
Testing instance 289 of 5889
Testing instance 290 of 5889
Testing instance 291 of 5889
Testing instance 292 of 5889
Testing instance 293 of 5889
Testing instance 294 of 5889
Testing instance 295 of 5889
Testing instance 296 of 5889
Testing instance 297 of 5889
Testing instance 298 of 5889
Testing instance 299 of 5889
Testing instance 300 of 5889
Testing instance 301 of 5889
Testing instance 302 of 5889
Testing instance 303 of 5889
Testing instance 304 of 5889
Testing instance 305 of 5889
Testing instance 306 of 5889
Testing instance 307 of 5889
Testing instance 308 of 5889
Testing instance 309 of 5889
Testing instance 310 of 5889
Testing instance 311 of 5889
Testing instance 312 of 5889
Testing instance 313 of 5889
Testing instance 314 of 5889
Testing instan

Testing instance 564 of 5889
Testing instance 565 of 5889
Testing instance 566 of 5889
Testing instance 567 of 5889
Testing instance 568 of 5889
Testing instance 569 of 5889
Testing instance 570 of 5889
Testing instance 571 of 5889
Testing instance 572 of 5889
Testing instance 573 of 5889
Testing instance 574 of 5889
Testing instance 575 of 5889
Testing instance 576 of 5889
Testing instance 577 of 5889
Testing instance 578 of 5889
Testing instance 579 of 5889
Testing instance 580 of 5889
Testing instance 581 of 5889
Testing instance 582 of 5889
Testing instance 583 of 5889
Testing instance 584 of 5889
Testing instance 585 of 5889
Testing instance 586 of 5889
Testing instance 587 of 5889
Testing instance 588 of 5889
Testing instance 589 of 5889
Testing instance 590 of 5889
Testing instance 591 of 5889
Testing instance 592 of 5889
Testing instance 593 of 5889
Testing instance 594 of 5889
Testing instance 595 of 5889
Testing instance 596 of 5889
Testing instance 597 of 5889
Testing instan

Testing instance 847 of 5889
Testing instance 848 of 5889
Testing instance 849 of 5889
Testing instance 850 of 5889
Testing instance 851 of 5889
Testing instance 852 of 5889
Testing instance 853 of 5889
Testing instance 854 of 5889
Testing instance 855 of 5889
Testing instance 856 of 5889
Testing instance 857 of 5889
Testing instance 858 of 5889
Testing instance 859 of 5889
Testing instance 860 of 5889
Testing instance 861 of 5889
Testing instance 862 of 5889
Testing instance 863 of 5889
Testing instance 864 of 5889
Testing instance 865 of 5889
Testing instance 866 of 5889
Testing instance 867 of 5889
Testing instance 868 of 5889
Testing instance 869 of 5889
Testing instance 870 of 5889
Testing instance 871 of 5889
Testing instance 872 of 5889
Testing instance 873 of 5889
Testing instance 874 of 5889
Testing instance 875 of 5889
Testing instance 876 of 5889
Testing instance 877 of 5889
Testing instance 878 of 5889
Testing instance 879 of 5889
Testing instance 880 of 5889
Testing instan

Testing instance 1126 of 5889
Testing instance 1127 of 5889
Testing instance 1128 of 5889
Testing instance 1129 of 5889
Testing instance 1130 of 5889
Testing instance 1131 of 5889
Testing instance 1132 of 5889
Testing instance 1133 of 5889
Testing instance 1134 of 5889
Testing instance 1135 of 5889
Testing instance 1136 of 5889
Testing instance 1137 of 5889
Testing instance 1138 of 5889
Testing instance 1139 of 5889
Testing instance 1140 of 5889
Testing instance 1141 of 5889
Testing instance 1142 of 5889
Testing instance 1143 of 5889
Testing instance 1144 of 5889
Testing instance 1145 of 5889
Testing instance 1146 of 5889
Testing instance 1147 of 5889
Testing instance 1148 of 5889
Testing instance 1149 of 5889
Testing instance 1150 of 5889
Testing instance 1151 of 5889
Testing instance 1152 of 5889
Testing instance 1153 of 5889
Testing instance 1154 of 5889
Testing instance 1155 of 5889
Testing instance 1156 of 5889
Testing instance 1157 of 5889
Testing instance 1158 of 5889
Testing in

...

Testing instance 4962 of 5889
Testing instance 4963 of 5889
Testing instance 4964 of 5889
Testing instance 4965 of 5889
Testing instance 4966 of 5889
Testing instance 4967 of 5889
Testing instance 4968 of 5889
Testing instance 4969 of 5889
Testing instance 4970 of 5889
Testing instance 4971 of 5889
Testing instance 4972 of 5889
Testing instance 4973 of 5889
Testing instance 4974 of 5889
Testing instance 4975 of 5889
Testing instance 4976 of 5889
Testing instance 4977 of 5889
Testing instance 4978 of 5889
Testing instance 4979 of 5889
Testing instance 4980 of 5889
Testing instance 4981 of 5889
Testing instance 4982 of 5889
Testing instance 4983 of 5889
Testing instance 4984 of 5889
Testing instance 4985 of 5889
Testing instance 4986 of 5889
Testing instance 4987 of 5889
Testing instance 4988 of 5889
Testing instance 4989 of 5889
Testing instance 4990 of 5889
Testing instance 4991 of 5889
Testing instance 4992 of 5889
Testing instance 4993 of 5889
Testing instance 4994 of 5889
Testing in

Testing instance 5236 of 5889
Testing instance 5237 of 5889
Testing instance 5238 of 5889
Testing instance 5239 of 5889
Testing instance 5240 of 5889
Testing instance 5241 of 5889
Testing instance 5242 of 5889
Testing instance 5243 of 5889
Testing instance 5244 of 5889
Testing instance 5245 of 5889
Testing instance 5246 of 5889
Testing instance 5247 of 5889
Testing instance 5248 of 5889
Testing instance 5249 of 5889
Testing instance 5250 of 5889
Testing instance 5251 of 5889
Testing instance 5252 of 5889
Testing instance 5253 of 5889
Testing instance 5254 of 5889
Testing instance 5255 of 5889
Testing instance 5256 of 5889
Testing instance 5257 of 5889
Testing instance 5258 of 5889
Testing instance 5259 of 5889
Testing instance 5260 of 5889
Testing instance 5261 of 5889
Testing instance 5262 of 5889
Testing instance 5263 of 5889
Testing instance 5264 of 5889
Testing instance 5265 of 5889
Testing instance 5266 of 5889
Testing instance 5267 of 5889
Testing instance 5268 of 5889
Testing in

Testing instance 5510 of 5889
Testing instance 5511 of 5889
Testing instance 5512 of 5889
Testing instance 5513 of 5889
Testing instance 5514 of 5889
Testing instance 5515 of 5889
Testing instance 5516 of 5889
Testing instance 5517 of 5889
Testing instance 5518 of 5889
Testing instance 5519 of 5889
Testing instance 5520 of 5889
Testing instance 5521 of 5889
Testing instance 5522 of 5889
Testing instance 5523 of 5889
Testing instance 5524 of 5889
Testing instance 5525 of 5889
Testing instance 5526 of 5889
Testing instance 5527 of 5889
Testing instance 5528 of 5889
Testing instance 5529 of 5889
Testing instance 5530 of 5889
Testing instance 5531 of 5889
Testing instance 5532 of 5889
Testing instance 5533 of 5889
Testing instance 5534 of 5889
Testing instance 5535 of 5889
Testing instance 5536 of 5889
Testing instance 5537 of 5889
Testing instance 5538 of 5889
Testing instance 5539 of 5889
Testing instance 5540 of 5889
Testing instance 5541 of 5889
Testing instance 5542 of 5889
Testing in

Testing instance 5784 of 5889
Testing instance 5785 of 5889
Testing instance 5786 of 5889
Testing instance 5787 of 5889
Testing instance 5788 of 5889
Testing instance 5789 of 5889
Testing instance 5790 of 5889
Testing instance 5791 of 5889
Testing instance 5792 of 5889
Testing instance 5793 of 5889
Testing instance 5794 of 5889
Testing instance 5795 of 5889
Testing instance 5796 of 5889
Testing instance 5797 of 5889
Testing instance 5798 of 5889
Testing instance 5799 of 5889
Testing instance 5800 of 5889
Testing instance 5801 of 5889
Testing instance 5802 of 5889
Testing instance 5803 of 5889
Testing instance 5804 of 5889
Testing instance 5805 of 5889
Testing instance 5806 of 5889
Testing instance 5807 of 5889
Testing instance 5808 of 5889
Testing instance 5809 of 5889
Testing instance 5810 of 5889
Testing instance 5811 of 5889
Testing instance 5812 of 5889
Testing instance 5813 of 5889
Testing instance 5814 of 5889
Testing instance 5815 of 5889
Testing instance 5816 of 5889
Testing in

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/bpic2011/xgboost_single_agg/instances/true_neg_bucket_1_.pickle'

In [12]:
len(feature_combiner.get_feature_names())

1041